### Maintenance

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
data = pd.read_csv("data/db_customers.csv")
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
data['order_delivered_customer_date'] = pd.to_datetime(data['order_delivered_customer_date'])

In [3]:
data.head()

,Unnamed: 0,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,payment_type,payment_installments,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,credit_card,1.0,18.12,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
1,1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,2.00,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
2,2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,18.59,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
3,3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,boleto,1.0,141.46,4,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,perfumery
4,4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,credit_card,3.0,179.12,5,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,auto


In [4]:
# Années
def year_data():
    data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
    year = []
    for i in data["order_purchase_timestamp"]:
        if i.year == 2017:
            year.append("2017")
        else:
            year.append("2018")
    return year

In [5]:
# Mois
def month_data():
    data['order_purchase_timestamp'] = data['order_purchase_timestamp'].astype(str)
    month = []
    for i in data["order_purchase_timestamp"]:
        if i[5:7] == "01":
            month.append("Janvier")
        if i[5:7] == "02":
            month.append("Fevrier")
        if i[5:7] == "03":
            month.append("Mars")
        if i[5:7] == "04":
            month.append("Avril")
        if i[5:7] == "05":
            month.append("Mai")
        if i[5:7] == "06":
            month.append("Juin")
        if i[5:7] == "07":
            month.append("Juillet")
        if i[5:7] == "08":
            month.append("Aout")
        if i[5:7] == "09":
            month.append("Septembre")
        if i[5:7] == "10":
            month.append("Octobre")
        if i[5:7] == "11":
            month.append("Novembre")
        if i[5:7] == "12":
            month.append("Decembre")   
    return month

In [6]:
# Ajout des colonnes mois & années
data['Mois']  = month_data()
data['Annee'] = year_data()

In [7]:
# Séparation du dataset (2017 & 2018)
data_2018 = data[data['Annee'] == "2018"]
data_2017 = data[data['Annee'] == "2017"]

## Modèle

In [8]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import adjusted_rand_score

In [9]:
drop          = ["Unnamed: 0", "order_delivered_customer_date", "customer_id", "order_id", "customer_unique_id", "Annee", "Mois", "order_purchase_timestamp", "order_status", "product_category_name_english"]
passthrough   = ["review_score", "payment_value", "payment_installments"]
cat           = ["payment_type", "customer_zip_code_prefix", "customer_state", "customer_city"]

In [10]:
preprocessing = make_column_transformer(
    (MinMaxScaler(),                      passthrough),
    ("drop",                              drop),
    (OrdinalEncoder(),                    cat),
    
)

In [11]:
# On récupère tous le dataset 2017 pour le train
preprocessed_array = preprocessing.fit_transform(data_2017)
X_train = pd.DataFrame(preprocessed_array)

In [12]:
def compare():
    data_dict=[]
    mois=['Janvier', 'Fevrier', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet',
       'Aout', 'Septembre', 'Octobre', 'Decembre']
    
    for i in mois:
        
        # échantillon mois Septembre 
        data = data_2018[data_2018['Mois'] == i]
        preprocessed_array = preprocessing.fit_transform(data)
        X_test = pd.DataFrame(preprocessed_array)
        X_test = X_test.dropna()

        kmeans = KMeans(n_clusters=3)

        # Mois (C1)
        c1 = kmeans.fit(X_train)
        predict_month = c1.predict(X_test)

        # 2017 (C1')     
        concat = pd.concat([X_train, X_test], axis=0)
        c1_ = kmeans.fit(concat)
        predict_2017 = c1_.predict(X_test)

        # Création du ARI     
        ari = adjusted_rand_score(predict_2017, predict_month)
        
        dictionnaire = {
            "Mois":i,
            "Ari":ari
        }
        
        data_dict.append(dictionnaire)     
    data_dict = pd.DataFrame(data_dict)
    return data_dict

In [17]:
compare()

,Mois,Ari
0,Janvier,0.960803
1,Fevrier,0.934091
